In [1]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier

import time

In [2]:
# On supprime les avertissements nous indiquant que l'on change les valeurs de notre jeu de données d'origine
pd.options.mode.chained_assignment = None

## Classification

In [3]:
# On charge nos données
df = pd.read_csv('data_preprocessed.csv')
df.drop(columns=['Unnamed: 0', 'index'], inplace=True)
df.set_index('SK_ID_CURR', inplace=True)

In [4]:
# On sépare en jeu de test et jeu d'entraînement
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]

In [5]:
# On regarde l'équilibre de nos classes
train_df['TARGET'].value_counts()

0.0    282682
1.0     24825
Name: TARGET, dtype: int64

In [6]:
# On prépare nos folds
X = train_df.drop('TARGET', axis = 1)
y = train_df['TARGET']
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)
cv = folds.split(X, y)
cv = list(cv)

## Baseline

In [7]:
t0 = time.time()

oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

# On sépare notre train en train et validation
for n_fold, (train_idx, valid_idx) in enumerate(cv):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
    
    clf = AdaBoostClassifier(random_state=24)
    clf.fit(train_x, train_y)
    
    oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
    sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits

    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))

t1 = time.time()
time_round = np.round(t1 - t0, 2)
print(f'Time : {time_round} secondes')

Fold  1 AUC : 0.771654
Fold  2 AUC : 0.777140
Fold  3 AUC : 0.771100
Fold  4 AUC : 0.777200


KeyboardInterrupt: 

## Gestion des classes

In [ ]:
results = []

In [10]:
t0 = time.time()

oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

# On sépare notre train en train et validation
for n_fold, (train_idx, valid_idx) in enumerate(cv):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]
    
    clf = AdaBoostClassifier(n_estimators=100, random_state=24)
    clf.fit(train_x, train_y)
    
    oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
    sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits

    roc_score = roc_auc_score(valid_y, oof_preds[valid_idx])
    results.append(roc_score)
    print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_score))

t1 = time.time()
time_round = np.round(t1 - t0, 2)
print(f'Time : {time_round} secondes')

Fold  1 AUC : 0.766544
Fold  2 AUC : 0.769772
Fold  3 AUC : 0.765211
Fold  4 AUC : 0.771102
Fold  5 AUC : 0.763635
Time : 2379.52 secondes


In [11]:
round(np.mean(results), 4)

0.7673